This notebooks is not a one off.

I keep coming back to it to make different version of the dataset.

In [14]:
import sys
sys.path.append("../scripts")
from fetch_data import DataLoader
from cleaning import CleanDataFrame
from visualization import Plotters
from processing import FeatureEngineering
from exploration import Analysis

In [15]:
cleaner = CleanDataFrame()
plotters = Plotters(w=8, h=8)
analyzer = Analysis()


In [20]:
DATASET_PATH = "../data/Debernardi et al 2020 data.csv"

In [21]:
df = DataLoader.read_csv(DATASET_PATH)

Pandas: CSV read from: ../data/Debernardi et al 2020 data.csv


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sample_id                590 non-null    object 
 1   patient_cohort           590 non-null    object 
 2   sample_origin            590 non-null    object 
 3   age                      590 non-null    int64  
 4   sex                      590 non-null    object 
 5   diagnosis                590 non-null    int64  
 6   stage                    199 non-null    object 
 7   benign_sample_diagnosis  208 non-null    object 
 8   plasma_CA19_9            350 non-null    float64
 9   creatinine               590 non-null    float64
 10  LYVE1                    590 non-null    float64
 11  REG1B                    590 non-null    float64
 12  TFF1                     590 non-null    float64
 13  REG1A                    306 non-null    float64
dtypes: float64(6), int64(2), o

For this first version, I will remove all the patient related attributes that don't relate to the disease and don't contribute to the generality of my model.
These will be columns like the `sample_id` which is unique for each patient.

- sample_id
- patient_cohort
- sample_origin
- stage
- benign_sample_diagnosis

In [23]:
col_to_drop = [
    "sample_id",
    "patient_cohort",
    "sample_origin",
    "stage",
    "benign_sample_diagnosis",
]
df = cleaner.drop_columns(df, col_to_drop)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            590 non-null    int64  
 1   sex            590 non-null    object 
 2   diagnosis      590 non-null    int64  
 3   plasma_CA19_9  350 non-null    float64
 4   creatinine     590 non-null    float64
 5   LYVE1          590 non-null    float64
 6   REG1B          590 non-null    float64
 7   TFF1           590 non-null    float64
 8   REG1A          306 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 41.6+ KB


In [24]:
analyzer.get_missing_entries_count(df)

(plasma_CA19_9    240
 REG1A            284
 dtype: int64,
 array(['plasma_CA19_9', 'REG1A'], dtype=object))

In [25]:
analyzer.percent_missing(df)

The dataset contains 9.87 % missing values.


# Save the data

In [26]:
df.to_csv(DATASET_PATH, index=False)